In [175]:
using Chain
using DataFrames
using CSV
using StatsBase
using Statistics
using Gadfly
using Flux

In [ ]:
struct DataLine
    number_columns
    categorical_columns
    log_columns
end

In [151]:
train_data = CSV.read("../data/train.csv", DataFrame)
train_data = select(train_data, Not(:Id))
first(train_data, 5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour
,Int64,String,String,Int64,String,String,String,String
1,60,RL,65,8450,Pave,NA,Reg,Lvl
2,20,RL,80,9600,Pave,NA,Reg,Lvl
3,60,RL,68,11250,Pave,NA,IR1,Lvl
4,70,RL,60,9550,Pave,NA,IR1,Lvl
5,60,RL,84,14260,Pave,NA,IR1,Lvl


In [109]:
function log_norm_skew(df, thres=0.7)
    num_cols = @chain begin
        names(df)
        filter(i -> eltype(df[!, i]) <: Number && skewness(df[!, i]) > 0.75, _)
    end
    for col in num_cols
        transform!(df, col => ByRow(x -> log2(x + 1)) => "log_$col")
        select!(df,  Not(col))
    end
    return df
end
log_norm_skew(df, thres=0.7) = log_norm_skew!(copy(df), thres)

log_norm_skew (generic function with 2 methods)

In [148]:
function categoricalize!(df)
    cols = @chain begin
        names(df)
        filter(i -> eltype(df[!, i]) <: AbstractString, _)
    end
    transforms = map(cols) do col
        values = unique(df[!, col])
        map(values) do value
            col => ByRow(isequal(value)) => Symbol("$(col)_$(value)")
        end
    end
    @chain begin
        transform!(df, transforms...)
        select!(_, Not(cols))
    end
end
categoricalize(df) = categoricalize!(copy(df))

categoricalize (generic function with 1 method)

In [116]:
function data_pipeline(df)
    @chain begin
        df
        log_norm_skew(_)
        categoricalize(_)
    end
end

data_pipeline (generic function with 1 method)

In [317]:
function model1(xtrain, ytrain)
    n = size(xtrain, 2)
    model = Dense(n, 1)
    data = @chain begin
        xtrain
        Matrix{Float32}(_)
        eachrow(_)
        collect(_)
        zip(_, ytrain[!, 1])
        collect(_)
    end
    opt = ADAM(1f-3)
    ps = params(model)
    loss(x, y) = Flux.Losses.mse(model(x), y)
    Flux.Optimise.train!(loss, ps, data, opt)
    model
end

model1 (generic function with 1 method)

In [318]:
m = model1(xtrain, ytrain)

Dense(838, 1)       # 839 parameters

In [322]:
# train_data, test_data = train_data[1:1000, :], train_data[1001:end, :]

In [323]:
xtrain, ytrain = let df = data_pipeline(train_data)
    select(df, Not("SalePrice")), select(df, "SalePrice");
end;

In [329]:
xtest, ytest = select(test_data, Not("SalePrice")), select(test_data, "SalePrice");

In [349]:
function predict(model, x)
    @chain begin
        x
        DataFrame(_)
        data_pipeline(_)
    end
#     x_ = data_pipeline(DataFrame(x))
#     input = [Float32(x_[name]) for name in names(x)]
#     model(input)
end

predict (generic function with 1 method)

In [348]:
predict(m, xtest[1, :])

LoadError: MethodError: no method matching names(::NamedTuple{(:MSSubClass, :MSZoning, :LotFrontage, :LotArea, :Street, :Alley, :LotShape, :LandContour, :Utilities, :LotConfig, :LandSlope, :Neighborhood, :Condition1, :Condition2, :BldgType, :HouseStyle, :OverallQual, :OverallCond, :YearBuilt, :YearRemodAdd, :RoofStyle, :RoofMatl, :Exterior1st, :Exterior2nd, :MasVnrType, :MasVnrArea, :ExterQual, :ExterCond, :Foundation, :BsmtQual, :BsmtCond, :BsmtExposure, :BsmtFinType1, :BsmtFinSF1, :BsmtFinType2, :BsmtFinSF2, :BsmtUnfSF, :TotalBsmtSF, :Heating, :HeatingQC, :CentralAir, :Electrical, Symbol("1stFlrSF"), Symbol("2ndFlrSF"), :LowQualFinSF, :GrLivArea, :BsmtFullBath, :BsmtHalfBath, :FullBath, :HalfBath, :BedroomAbvGr, :KitchenAbvGr, :KitchenQual, :TotRmsAbvGrd, :Functional, :Fireplaces, :FireplaceQu, :GarageType, :GarageYrBlt, :GarageFinish, :GarageCars, :GarageArea, :GarageQual, :GarageCond, :PavedDrive, :WoodDeckSF, :OpenPorchSF, :EnclosedPorch, Symbol("3SsnPorch"), :ScreenPorch, :PoolAr